In [1]:
#!pip install kaggle

In [5]:
!kaggle competitions download -c titanic



  0%|          | 0.00/34.1K [00:00<?, ?B/s]
100%|██████████| 34.1K/34.1K [00:00<00:00, 110KB/s]


In [6]:
import zipfile as zf 
files = zf.ZipFile("titanic.zip",'r')
files.extractall('dataset')
files.close()

In [7]:
! del titanic.zip

In [43]:
import pandas as pd 
df = pd.read_csv('dataset/train.csv')
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [44]:
print(df.columns)
print(df.info())

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB
None


## Analisis de muestras sobre un Dataframe con df.groupby().__iter__()

usualmente es complicado explorar un conjunto de datos linea por linea, Si uno compara con Excel. Un truco bastante bueno es utilizar ctrl + Enter (comando) de manera de iterar una vista diferente en cada acción del comando.

Vamos a crear un generador con groupby() <br>
generador = df.groupby(['identificador'])."__iter__"

Posteriormente vamos a seleccionar el Grupo y vamos a iterar sobre este grupo.<br>
grupo_id, grupo_data = generador."__next()__"

se imprime el Grupo y los Datos


In [45]:
tickets_compartidos_con_2_o_mas = df['Ticket'].value_counts().loc[
    (df['Ticket'].value_counts()>2)
    ].index.to_list()


In [46]:
# vamos a crear el generador para poder iterar a traves de lo tickets
generator = df.loc[df['Ticket'].isin(tickets_compartidos_con_2_o_mas)].groupby('Ticket').__iter__()


In [47]:
# Se mira a de manera individual por cada Grupo usando control + Enter
group_id, group_data = generator.__next__()
print(group_id)
group_data

110152


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
257,258,1,1,"Cherry, Miss. Gladys",female,30.0,0,0,110152,86.5,B77,S
504,505,1,1,"Maioni, Miss. Roberta",female,16.0,0,0,110152,86.5,B79,S
759,760,1,1,"Rothes, the Countess. of (Lucy Noel Martha Dye...",female,33.0,0,0,110152,86.5,B77,S


## Combinación de multiples operaciones usando una cadea de metodos
Ejemplo: <br>
1.- Agregar una nueva columns (.merge) <br>
2.- contar la proporcion de mujeres pasajeras (.apply(female_proportion)) <br>
3.- Agrupar uno o mas pasajeros con cantidad de tickets (df.ticket.value_counts()>1) <br>
4.- obtener el mismo n° ticket 

In [48]:
def female_proportion(dataframe):
    return (dataframe.Sex =='female')/len(dataframe)

female_proportion(df)

0      0.000000
1      0.001122
2      0.001122
3      0.001122
4      0.000000
         ...   
886    0.000000
887    0.001122
888    0.001122
889    0.000000
890    0.000000
Name: Sex, Length: 891, dtype: float64

In [49]:
df.merge(
    # la creacion de female proportion se une con el dataframe segun el nro de ticket
    df.loc[ #se seleccionan todos los ticket mayores a uno que tiene valores y se agrupan por ticket, reseteado el indice 
        df.Ticket.isin( #se obtiene solo un True si tiene un valor
            df.Ticket.value_counts().loc[ #filtra todos los tickets que tienen mas que 1 y entrega el indice (index['347082', '1601', 'CA. 2343', 'CA 2144', '3101295', '347088'...])
                df.Ticket.value_counts()>1 #entrega un Bool si es mayor que 1
            ].index
        )
    ].groupby('Ticket').apply(female_proportion) \
    .reset_index().rename(columns={0:'proportion_female'}) #  reseteado el indice  y cambiando el nombre a la columnta 0
    ,how='left', on='Ticket'
)


,PassengerId,Survived,Pclass,Name,Sex_x,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,level_1,Sex_y
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,NaN,NaN
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,NaN,NaN
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,NaN,NaN
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,3.0,0.5
4,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,137.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1588,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S,NaN,NaN
1589,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S,783.0,0.0
1590,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S,888.0,0.5
1591,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C,NaN,NaN


## uso de Sklearn_pandas

es el Reemplazo de ColumnTransforme con un amigable DataFramMapper.

In [2]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn_pandas import DataFrameMapper

ModuleNotFoundError: No module named 'sklearn_pandas'

In [53]:
import sklearn

In [54]:
sklearn.__version__

'0.22.2.post1'

In [ ]:
import sklearn_